In [1]:
import pandas as pd
import numpy as np

#load file
sale = pd.read_csv('sales.csv')
price = pd.read_csv('prices.csv')
print(sale)
print(price)

     product_id     ordered_at  quantity_ordered
0       3998909  9/18/18 17:51                 1
1       3998909  9/18/18 12:52                 1
2       3998909  9/18/18 11:33                 1
3       3998909  9/18/18 18:47                 1
4       3998909  9/18/18 17:36                 1
..          ...            ...               ...
170     4085861  9/13/18 23:29                 1
171          64  9/11/18 19:32                 1
172          64  9/14/18 20:10                 1
173          64  9/12/18 16:00                 1
174          64  9/12/18 13:23                 1

[175 rows x 3 columns]
    product_id  old_price  new_price     updated_at
0           64     270000     239000  9/10/18 16:37
1      3954203      60000      64000  9/11/18 11:54
2      3954203      60500      57500  9/17/18 22:59
3      3954203      64000      60500   9/15/18 3:49
4      3954203      68800      60000  9/10/18 16:32
5      3998909      15500      16500   9/16/18 5:09
6      3998909      1700

In [2]:
#convert ordered_at, updated_at to datetime object (using pd.to_datetime)
sale['ordered_at'] = pd.to_datetime(sale['ordered_at'])
price['updated_at'] = pd.to_datetime(price['updated_at'])

In [3]:
#def function that return the cloest date that price update
def closest_update_date (id, order_date):
    new_date = price['updated_at'][price['product_id'] == id]
    if order_date < new_date.min():
       closest_date = new_date.min()
    else :
       closest_date = new_date[order_date > new_date].max()                            
    return closest_date

In [4]:
#def function that return closest price base on productid and order_date
def return_closest_price(id, order_date):
    day = closest_update_date (id, order_date)
    if order_date < day :
       closest_price = price['old_price'][(price['updated_at'] == day) & (price['product_id'] == id)].values[0]
    else :
       closest_price = price['new_price'][(price['updated_at'] == day) & (price['product_id'] == id)].values[0]                              
    return closest_price

In [5]:
#Find price corresponding each row and append to update_price array.
update_price=[]
for i in np.arange(len(sale)):
    updated_price = return_closest_price(sale.iloc[i]['product_id'], sale.iloc[i]['ordered_at'])
    update_price.append(updated_price)

In [6]:
#create and calculate price and revenue column
sale["price"] = update_price
sale["revenue"] = sale['price'] * sale['quantity_ordered']

In [7]:
#export to csv file
print(sale)
sale.to_csv("revenue.csv")

     product_id          ordered_at  quantity_ordered   price  revenue
0       3998909 2018-09-18 17:51:00                 1   16500    16500
1       3998909 2018-09-18 12:52:00                 1   16500    16500
2       3998909 2018-09-18 11:33:00                 1   16500    16500
3       3998909 2018-09-18 18:47:00                 1   16500    16500
4       3998909 2018-09-18 17:36:00                 1   16500    16500
..          ...                 ...               ...     ...      ...
170     4085861 2018-09-13 23:29:00                 1   62500    62500
171          64 2018-09-11 19:32:00                 1  239000   239000
172          64 2018-09-14 20:10:00                 1  239000   239000
173          64 2018-09-12 16:00:00                 1  239000   239000
174          64 2018-09-12 13:23:00                 1  239000   239000

[175 rows x 5 columns]


In [8]:
print(sale.groupby('product_id').agg({'revenue':sum}))

            revenue
product_id         
64           956000
3954203      877500
3998909      280500
4085861     8247500
